### Collect Reddit Data - ANLY 590 Project
#### November 13, 2018

***

#### Prep

In [1]:
# set working directory
import os
import sys
path = '/Users/kgedney/Documents/georgetown/anly590/author-id-project'
os.chdir(path)

In [2]:
# add API info
client_id     = 'my client id'
client_secret = 'my client secret'
user_agent    = 'deeplearning'
username      = 'my user name'
password      = ''

In [3]:
# import packages
import praw
import pandas as pd
import datetime as dt
from tqdm import tqdm

In [4]:
# set reddit credentials
reddit = praw.Reddit(client_id     = client_id, \
                     client_secret = client_secret, \
                     user_agent    = user_agent, \
                     username      = username, \
                     password      = password)

***

####  Scrape Comments

In [13]:
# import usernames
midterms = pd.read_csv('Midterms.csv')
midterms.head()
midterms = midterms.iloc[1:]
print(len(midterms['author'].unique()))

421


In [6]:
# intialize dictionary
usernames     = midterms['author'].unique()
comments_dict = {'author': [], 
                 'body': []}

In [7]:
# run scrape
for username in tqdm(usernames):
    try:
        for comment in reddit.redditor(username).comments.new(limit=None):
            comments_dict['author'].append(username)
            comments_dict['body'].append(comment.body.split('\n', 1))
    except KeyboardInterrupt:
        raise
    except:
        print('error at %s' % username, file=sys.stderr) 

100%|██████████| 422/422 [1:28:20<00:00, 12.18s/it]


In [23]:
# create df
df = pd.DataFrame(comments_dict)
df.head()

(353091, 2)


,author,body
0,AutoModerator,[Your submission has been removed from /r/sams...
1,AutoModerator,"[, As a reminder, this subreddit [is for civil..."
2,AutoModerator,[Your post have been removed as it appears tha...
3,AutoModerator,[Your post was removed because it contained a ...
4,AutoModerator,[Your submission has been removed. Stories in ...


In [37]:
# remove AutoModerator posts
df = df[df['author'] != 'AutoModerator'].reset_index(drop=True)
print(df.shape)
df.head()

(352096, 2)


,author,body
0,BlakeIsBlake,[I just went through this process myself. What...
1,BlakeIsBlake,[Dude. No.]
2,BlakeIsBlake,[I was still skeptical even after reading your...
3,BlakeIsBlake,"[> NYU School of Continuing Education, \nYes. ..."
4,BlakeIsBlake,[Still selling? Interested]


In [25]:
# save as csv
df.to_csv('raw_data.csv')

***

#### Exploratory Analysis

In [ ]:
# read in data
df = pd.read_csv('raw_data.csv')

In [32]:
# average of 838 posts per author
df.groupby('author').count().mean()

body    838.32381
dtype: float64

In [33]:
# 420 unique authors
len(set(df['author']))

420

In [51]:
# combine back lists
df['whole_body'] = df.body.apply(lambda x: ' '.join(x))

In [57]:
# create new columns for word count and character count
df['num_words'] = df.whole_body.apply(lambda x: len(x.split(' ')))
df['num_chars'] = df.whole_body.apply(lambda x: len(x))

In [61]:
# average words (just on a basic tokenization)
df['num_words'].mean()

43.83390041352359

In [63]:
# average characters
df['num_chars'].mean()

251.633741934018

In [64]:
df['num_chars'].max()

9999

In [65]:
df['num_words'].max()

2048

***

### Old Stuff

In [ ]:
subreddit = (reddit.subreddit('technology'))

In [ ]:
for submission in subreddit(limit=None):
    print(submission.title, submission.id)

In [74]:
# print comments
comments_dictionary = {}
i = 0

for comment in tqdm((reddit.subreddit('technology').comments(limit=None))):
    comments_dictionary[i] = (comment.body, comment.author, comment.author_fullname)
    i = i + 1
    #print(comment.body, comment.author, comment.author_fullname)

966it [00:12, 77.59it/s]


In [73]:
len(comments_dictionary)

967

In [66]:
# vars(comment)